<a href="https://colab.research.google.com/github/SashankRaja/source-sprint-event/blob/main/Lemmatimised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

In [ ]:
import zipfile
import os

# File paths
json_file = 'kaggle.json'  # Your JSON file
zip_file = 'kaggle.zip'    # The desired output ZIP file

# Create a ZIP file and add the JSON file to it
with zipfile.ZipFile(zip_file, 'w') as zipf:
    zipf.write(json_file, os.path.basename(json_file))

print(f'{json_file} has been compressed to {zip_file}')


In [5]:
from zipfile import ZipFile
file_name = "kaggle.zip"

with ZipFile(file_name, 'r') as zip: #opening the file in read mode
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [6]:
#Importing the dependencies
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer #to reduce a word to its root word
from sklearn.feature_extraction.text import TfidfVectorizer  #we cannot directly feed texual data to an ML model first we should convert it to numeric thats why we use this
from sklearn.model_selection import train_test_split #to split data into training data and testing data
from sklearn.linear_model import LogisticRegression #alogorithm used
from sklearn.metrics import accuracy_score  #to check accuracy of our model
from nltk.stem import WordNetLemmatizer

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
print(stopwords.words('english'))
#stopwords: they are those words that do not add any proper meaning to the sentences, i.e the ML model does not these words for processing
#the contextual importance of these words are not that important
#we do this in order to remove all the stopwords from our tweets reducing its size therefore dealing with the data more effieciently.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
#Loading the csv file into pandas library
twitter_data=pd.read_csv('/content/sentiment140.zip',encoding='ISO-8859-1')

In [10]:
twitter_data.shape
twitter_data.head()
column_name=['target','ids','date','flag','user','text']
twitter_data.columns=column_name #assinging the coulmn names to each coulmn
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#checking the missing values if any in the dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [12]:
twitter_data['target'].value_counts()

,count
target,
4,800000
0,799999


In [13]:
twitter_data.replace([4,0],value=[1,0],inplace=True)

In [14]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Compile regex pattern to remove non-alphabetic characters once
pattern = re.compile('[^a-zA-Z]')

# Convert stopwords list to a set for faster lookup
english_stopwords = set(stopwords.words('english'))

# Initialize PorterStemmer
port_stemmer = PorterStemmer()

# Optimized function for stemming
def stemming(content):
    # Lowercase, remove non-alphabet characters, and split in one go
    stemmed_content = re.sub(pattern, ' ', content).lower().split()

    # Stemming and removing stopwords in a single list comprehension
    return ' '.join([port_stemmer.stem(word) for word in stemmed_content if word not in english_stopwords])

# Assuming `twitter_data` is a DataFrame with a 'text' column
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)


In [15]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,kwesidei whole crew


In [16]:
from tkinter.constants import X
#let us now separate the data and the target
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values
print(X)
print(Y)

['upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound'
 'whole bodi feel itchi like fire' ... 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
[0 0 0 ... 1 1 1]


In [17]:
#splitting data into training and testing
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2) #test_size 0.2 means 20% is used for testing the model and ret 80% is used to train it
#stratify Y means we want an equal proportion of positive and negative tweets

In [18]:
#let us perfom feature extraction using vectorisiation
#i.e we are converting the texual data into numeric values
vectorizer=TfidfVectorizer()#this is what converts the text into numeric data

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)
#vectorising the training and testing data

In [20]:
#training the ML model
Model=LogisticRegression(max_iter=10000)
Model.fit(X_train,Y_train) #the model will now learn/try to map what all words consitute a positive tweet and what all do for a negative tweet

LogisticRegression(max_iter=10000)

In [21]:
#let us check the accuracy score of our model now
X_train_prediction=Model.predict(X_train)#after model is done learning from training data u give the X train and ask it to predict if its positive or negative
training_data_accuracy=accuracy_score(Y_train,X_train_prediction)#model checks how far its predcition is from the actual score
print(training_data_accuracy)

0.81030141429798


In [22]:
#lets check accuraty with test data now
X_test_prediction=Model.predict(X_test)
test_data_accuracy=accuracy_score(Y_test,X_test_prediction)
print(test_data_accuracy)

0.777909375


In [23]:
#Now that we already trained our ML model instead of training the model again and again repeatedly before running our code we can just directly
#save our trained model using the python pickle library
import pickle
filename='trained_model.sav'
pickle.dump(Model,open(filename,'wb'))#pickle.dump means ur dumping all the model parameters and everythign it has learned and wb means write binary as we know


In [24]:
#now lets see how to use the saved model for future predictions
#let us load the model
loaded_model=pickle.load(open(filename,'rb'))#rb means read binary

In [25]:
X_new=X_test[2]
print(Y_test[2])
prediction=loaded_model.predict(X_new)
print(prediction)
if prediction[0]==0:
  print('Negative')
else:
  print('Positive')

0
[0]
Negative


In [26]:
#Let us also check using the naive bias algorithm
from sklearn.naive_bayes import MultinomialNB
X=twitter_data['text'].values
Y=twitter_data['target'].values
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)
Model=MultinomialNB()
Model.fit(X_train,Y_train)
accuracy_score(Y_test,Model.predict(X_test))

0.7733625

In [27]:
#let us now check if there is any change in perfermance if instead of stemming we perform Lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Function to preprocess and lemmatize text
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove non-alphabetic characters
    words = text.lower().split()  # Convert to lowercase and split words
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Apply lemmatization
    return ' '.join(words)

# Sample data
X=twitter_data['text'].values

# Preprocess and lemmatize the text
X_preprocessed = [preprocess_text(sentence) for sentence in X]

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [28]:
#Let us also check using the naive bias algorithm(with Lemmatization)
from sklearn.naive_bayes import MultinomialNB
X=X_preprocessed
Y=twitter_data['target'].values
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)
Model=MultinomialNB()
Model.fit(X_train,Y_train)
accuracy_score(Y_test,Model.predict(X_test))

0.761484375

In [29]:
#Normal Logistic Regression with Lemmatization
model=LogisticRegression(max_iter=10000)
model.fit(X_train,Y_train)
accuracy_score(Y_test,model.predict(X_test))

0.782190625

In [30]:
accuracy_score(Y_train,Model.predict(X_train))

0.8263412705791177